In [1]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import ion
import matplotlib.dates as mdates
%matplotlib tk
import numpy as np
from poliastro.bodies import Sun
from poliastro.core.iod import izzo
from datetime import datetime
from tqdm import tqdm
import numpy as np

In [2]:
targets = ['SUN', 'EARTH', 'MARS', 'SATELLITE']

fig = plt.figure()
ax = fig.add_subplot(projection='3d')

ax.set_xlabel('X (AU)')
ax.set_ylabel('Y (AU)')
ax.set_zlabel('Z (AU)')
ax.set_title('Orbit in AU Space')

all_px, all_py, all_pz = [], [], []

for target in targets:
    try :
        data = np.genfromtxt(f'simulation_results/{target}.csv', delimiter=',', skip_header=1)
    except :
        data = np.genfromtxt(f'traces/{target}.csv', delimiter=',', skip_header=1)
        
    px, py, pz = data[:, 1], data[:, 2], data[:, 3]
    
    all_px.extend(px)
    all_py.extend(py)
    all_pz.extend(pz)
    
    ax.plot(px, py, pz, label=target)  # Add label for better visualization

# Compute overall range
all_px, all_py, all_pz = np.array(all_px), np.array(all_py), np.array(all_pz)

max_range = max(all_px.ptp(), all_py.ptp(), all_pz.ptp()) / 2.0
mid_x = (all_px.max() + all_px.min()) / 2.0
mid_y = (all_py.max() + all_py.min()) / 2.0
mid_z = (all_pz.max() + all_pz.min()) / 2.0

# Set the global limits
ax.set_xlim(mid_x - max_range, mid_x + max_range)
ax.set_ylim(mid_y - max_range, mid_y + max_range)
ax.set_zlim(mid_z - max_range, mid_z + max_range)

ax.legend()  # Show labels

#plt.show()
print('done')

3

In [3]:
def load_data(host : str, target : str) :
    
    try :
        dates = np.genfromtxt(f'simulation_results\{host}.csv',delimiter=',',usecols=[0],skip_header=True,dtype=str) 

        host_positions = np.genfromtxt(f'simulation_results\{host}.csv',delimiter=',',usecols=[1,2,3],skip_header=True) 
        host_velocities = np.genfromtxt(f'simulation_results\{host}.csv',delimiter=',',usecols=[4,5,6],skip_header=True)

        target_positions = np.genfromtxt(f'simulation_results\{target}.csv',delimiter=',',usecols=[1,2,3],skip_header=True) 
        target_velocities = np.genfromtxt(f'simulation_results\{target}.csv',delimiter=',',usecols=[4,5,6],skip_header=True)  
    
    except : 
        dates = np.genfromtxt(f'traces\{host}.csv',delimiter=',',usecols=[0],skip_header=True,dtype=str) 

        host_positions = np.genfromtxt(f'traces\{host}.csv',delimiter=',',usecols=[1,2,3],skip_header=True) 
        host_velocities = np.genfromtxt(f'traces\{host}.csv',delimiter=',',usecols=[4,5,6],skip_header=True)

        target_positions = np.genfromtxt(f'traces\{target}.csv',delimiter=',',usecols=[1,2,3],skip_header=True) 
        target_velocities = np.genfromtxt(f'traces\{target}.csv',delimiter=',',usecols=[4,5,6],skip_header=True)  
        
    return dates, host_positions, host_velocities, target_positions, target_velocities

In [4]:
dates, host_positions, host_velocities, target_positions, target_velocities = load_data('EARTH_TRACE','MARS_TRACE')
dates = [datetime.strptime(date, "%A %B %d %Y %H:%M:%S").timestamp() for date in dates]
dates = np.array(dates)
t0 = dates[0]
dates = dates - t0

elements = dates.shape[0] 
resolution = 0.0001
interval = elements / (elements * resolution)

launch_from  = (0) *24*60*60
launch_until = (5*365) *24*60*60
launch_times = np.extract(dates >= launch_from,dates)
launch_times = np.extract(dates <= launch_until,launch_times)

mask = np.isin(dates, launch_times)
launch_positions = host_positions[mask]
launch_velocities = host_velocities[mask]

arrive_after  = 100 *24*60*60 
arrive_before = (900) *24*60*60

lower_bound = min(launch_times) + arrive_after
upper_bound = max(launch_times) + arrive_before 

arrival_times = np.extract(dates >= lower_bound,dates)
arrival_times = np.extract(dates <= upper_bound,arrival_times)

mask = np.isin(dates, arrival_times)
arrival_positions = target_positions[mask]
arrival_velocities = target_velocities[mask]

launch_times += t0
arrival_times += t0

In [5]:
ax = plt.figure().add_subplot(projection='3d')
ax.plot(host_positions[:,0],host_positions[:,1],host_positions[:,2],color='steelblue')
ax.plot(launch_positions[:,0],launch_positions[:,1],launch_positions[:,2],color='red')

ax.plot(target_positions[:,0],target_positions[:,1],target_positions[:,2],color='steelblue')
ax.plot(arrival_positions[:,0],arrival_positions[:,1],arrival_positions[:,2],color='red')

a = np.argmin(launch_times)
b = np.argmax(launch_times)
ax.scatter(launch_positions[a][0],launch_positions[a][1],launch_positions[a][2],color='steelblue')
ax.scatter(launch_positions[b][0],launch_positions[b][1],launch_positions[b][2],color='red')

print('fuck off retard')

fuck off retard


In [6]:
final = []

step = 100  # Change step size to control skipping rate
total_iterations = len(launch_positions[::step]) * len(target_positions[::step])

with tqdm(total=total_iterations, desc="Processing", unit="iter") as pbar:
    for lpos, lt, lv in zip(launch_positions[::step], launch_times[::step], launch_velocities[::step]):
        for tpos, tt, tv in zip(arrival_positions[::step], arrival_times[::step], arrival_velocities[::step]):
            
            tof = tt - lt
            
            try:
                v_i, _ = izzo(Sun.k, lpos, tpos, tof, 0, True, True, 150, 1e-8)
                if np.linalg.norm(v_i - lv)/1000 <= 7.81 : 
                    final.append([lt, tt, np.linalg.norm(v_i - lv)/1000])
                else : 
                    final.append([lt, tt, np.nan])
                    
            except:
                v_i = lv
                final.append([lt, tt, np.nan])

            pbar.update(1)  # Update progress bar for each iteration

final = np.array(final)

Processing:   0%|          | 0/2493288 [00:00<?, ?iter/s]

Processing:  81%|████████▏ | 2026524/2493288 [00:34<00:07, 59521.52iter/s] 


In [7]:
lt_vals = np.unique(final[:, 0])  # Unique launch times
tt_vals = np.unique(final[:, 1])  # Unique arrival times

lt_vals = [datetime.fromtimestamp(t) for t in lt_vals]
tt_vals = [datetime.fromtimestamp(t) for t in tt_vals]

v_grid = np.array(final[:, 2].reshape(len(lt_vals), len(tt_vals))).astype(np.float64)  

plt.figure(figsize=(8,6))
c = plt.contourf(lt_vals, tt_vals, (v_grid.T),levels=15)  
plt.colorbar(c,label=r'Required $\Delta$V Impulse Burn [km/s]')  

launch_start_date = datetime(2026, 8, 1) 
launch_end_date = datetime(2027, 2, 28)    

arrival_start_date = datetime(2026, 7, 1) 
arrival_end_date = datetime(2028, 5, 1)    

plt.xlim(launch_start_date, launch_end_date) 
plt.ylim(arrival_start_date, arrival_end_date)

plt.gca().xaxis.set_minor_locator(mdates.DayLocator(interval=6))  # Subticks every 5 days on x-axis
plt.gca().yaxis.set_minor_locator(mdates.DayLocator(interval=8))  # Subticks every 5 days on y-axis

plt.gca().tick_params(axis='x', which='minor', length=2, color='gray')  # Customize minor x-axis ticks
plt.gca().tick_params(axis='y', which='minor', length=2, color='gray')  # Customize minor y-axis ticks

plt.gca().yaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))  # Format as 'Jan 2025'
plt.gca().yaxis.set_major_locator(mdates.MonthLocator(interval=5))  # Show every month
plt.yticks(fontsize=7, rotation=45)  # Rotate labels for better visibility

plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))  # Format as 'Jan 2025'
plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=1))  # Show every month
plt.xticks(fontsize=7, rotation=45)  # Rotate labels for better visibility

plt.xlabel('Launch Date')
plt.ylabel('Arrival Date')

Text(0, 0.5, 'Arrival Date')